# WonderTrader + wtpy 完整框架体系与开发指南

本文档详细阐述：
1. **wtpy 的完整框架体系**
2. **WonderTrader 与 wtpy 的协作方式**
3. **基于 WonderTrader + wtpy 的完整开发流程**

---

# 第一部分：wtpy 完整框架体系

## 1.1 整体架构概览

wtpy 是构建在 WonderTrader C++ 核心之上的 Python 应用层框架，采用分层架构设计：

```
┌─────────────────────────────────────────────────────────┐
│              应用层 (Application Layer)                 │
│  ┌──────────┐  ┌──────────┐  ┌──────────┐  ┌────────┐│
│  │ 策略开发  │  │ 回测分析  │  │ 实时监控  │  │ 数据工具││
│  │  CTA/HFT │  │WtBtAnalyst│  │WtMonSvr  │  │DataHelper││
│  └──────────┘  └──────────┘  └──────────┘  └────────┘│
└─────────────────────────────────────────────────────────┘
                        ↕
┌─────────────────────────────────────────────────────────┐
│            Python 封装层 (Wrapper Layer)                │
│  ┌──────────┐  ┌──────────┐  ┌──────────┐  ┌────────┐│
│  │WtWrapper │  │WtBtWrapper│  │WtDtWrapper│  │WtExecApi││
│  │实盘引擎  │  │ 回测引擎   │  │ 数据引擎  │  │执行器  ││
│  └──────────┘  └──────────┘  └──────────┘  └────────┘│
└─────────────────────────────────────────────────────────┘
                        ↕ (ctypes/C接口)
┌─────────────────────────────────────────────────────────┐
│          C++ 核心层 (WonderTrader Core)                  │
│  ┌──────────┐  ┌──────────┐  ┌──────────┐  ┌────────┐│
│  │WtPorter  │  │WtBtPorter│  │WtDtPorter│  │WtMsgQue││
│  │实盘C接口 │  │回测C接口  │  │数据C接口  │  │消息队列 ││
│  └──────────┘  └──────────┘  └──────────┘  └────────┘│
│  ┌────────────────────────────────────────────────────┐ │
│  │  WtCore (CTA/HFT/SEL引擎)  +  WtBtCore (回测引擎) │ │
│  └────────────────────────────────────────────────────┘ │
└─────────────────────────────────────────────────────────┘
```

## 1.2 核心模块详解

### 1.2.1 wrapper 子模块 - C++ 底层对接层

**功能定位**：作为 Python 与 C++ 核心的桥梁，通过 ctypes 调用 C 接口动态库

**核心文件**：

| 文件 | 功能 | 对应的C++动态库 |
|------|------|----------------|
| `WtWrapper.py` | 实盘交易引擎包装器 | `WtPorter.dll/.so` |
| `WtBtWrapper.py` | 回测引擎包装器 | `WtBtPorter.dll/.so` |
| `WtDtWrapper.py` | 数据组件包装器 | `WtDtPorter.dll/.so` |
| `WtDtServoApi.py` | 数据伺服服务API | `WtDtServo.dll/.so` |
| `WtExecApi.py` | 独立执行器API | `WtExecMon.dll/.so` |
| `WtMQWrapper.py` | 消息队列包装器 | `WtMsgQue.dll/.so` |
| `WtDtHelper.py` | 数据辅助工具 | `WtDtHelper.dll/.so` |
| `ContractLoader.py` | 合约加载器 | `CTPLoader.dll/.so` |

**工作机制**：
- 使用 `ctypes.cdll.LoadLibrary()` 加载 C++ 动态库
- 通过 `restype` 和 `argtypes` 定义函数签名
- 提供 Python 友好的接口封装，隐藏底层 C 接口细节
- 单例模式确保全局唯一实例

**典型调用流程**：
```python
# 1. 加载动态库
self.api = cdll.LoadLibrary(_path)

# 2. 设置函数签名
self.api.cta_enter_long.argtypes = [c_ulong, c_char_p, c_double, ...]
self.api.cta_enter_long.restype = c_bool

# 3. Python层调用
self.api.cta_enter_long(strategy_id, code.encode(), qty, ...)
```

### 1.2.2 引擎模块 - 策略执行核心

#### WtEngine.py - 实盘交易引擎

**功能**：
- 管理 CTA/HFT/SEL 策略的初始化和运行
- 配置管理（合约、品种、交易时段、风控等）
- 策略生命周期管理（初始化、计算、下单、平仓等）
- 扩展模块注册（自定义解析器、执行器、数据加载器）

**核心方法**：
- `init()`: 初始化引擎，加载配置
- `add_cta_strategy()`: 添加 CTA 策略
- `add_hft_strategy()`: 添加 HFT 策略  
- `add_sel_strategy()`: 添加 SEL 策略
- `run()`: 启动引擎运行
- `stop()`: 停止引擎

#### WtBtEngine.py - 回测引擎

**功能**：
- 历史数据回放和策略回测
- 支持 CTA/HFT/SEL 策略的回测
- 回测结果数据输出
- 单步执行模式（用于强化学习训练）

**核心方法**：
- `init()`: 初始化回测引擎
- `add_strategy()`: 添加回测策略
- `engine.config_backtest()`: 配置回测参数（时间范围、初始资金等）
- `engine.run_backtest()`: 执行回测
- `engine.step()`: 单步执行（用于RL训练）

#### WtDtEngine.py - 数据引擎

**功能**：
- 管理行情数据接收和存储
- 支持多种行情源（CTP、XTP、OES等）
- 历史数据落地和读取
- 扩展数据解析器支持

**核心方法**：
- `init()`: 初始化数据引擎
- `add_parser()`: 添加行情解析器
- `run_datakit()`: 启动数据接收和存储

### 1.2.3 策略上下文模块 - 策略运行时环境

#### CtaContext.py - CTA策略上下文

**功能**：为 CTA 策略提供完整的运行时环境，是策略唯一可以访问的对象

**接口分类**：

**时间接口**：
- `stra_get_date()`: 获取当前日期
- `stra_get_time()`: 获取当前时间
- `stra_get_tdate()`: 获取当前交易日

**数据接口**：
- `stra_get_bars()`: 获取K线数据（返回 WtNpKline）
- `stra_get_ticks()`: 获取Tick数据（返回 WtNpTicks）
- `stra_get_position()`: 获取持仓数量
- `stra_get_position_profit()`: 获取持仓盈亏
- `stra_get_fund_data()`: 获取资金数据

**交易接口**：
- `stra_enter_long()`: 开多仓
- `stra_enter_short()`: 开空仓
- `stra_exit_long()`: 平多仓
- `stra_exit_short()`: 平空仓
- `stra_set_position()`: 设置目标仓位

**工具接口**：
- `stra_log_text()`: 输出日志
- `stra_save_userdata()`: 保存用户数据
- `stra_load_userdata()`: 加载用户数据
- `stra_get_comminfo()`: 获取品种信息
- `stra_get_sessinfo()`: 获取交易时段信息

#### HftContext.py - HFT策略上下文

**功能**：为高频交易策略提供接口，支持逐笔成交、委托队列等深度数据

**特色接口**：
- `stra_get_order_queue()`: 获取委托队列
- `stra_get_order_detail()`: 获取逐笔委托
- `stra_get_transaction()`: 获取逐笔成交
- `stra_subscribe_order_queue()`: 订阅委托队列
- `stra_subscribe_order_detail()`: 订阅逐笔委托

#### SelContext.py - 选股策略上下文

**功能**：为选股策略提供接口，主要用于多标的选择

**特色接口**：
- `stra_get_stock_list()`: 获取股票列表
- `stra_get_position_list()`: 获取持仓列表
- `stra_set_position()`: 设置目标仓位（多标的）

### 1.2.4 策略基类模块 - StrategyDefs.py

**功能**：定义所有策略类型的基类接口

**基类定义**：

#### BaseCtaStrategy - CTA策略基类

**回调函数**：
- `on_init(context)`: 策略初始化
- `on_session_begin(context, curTDate)`: 交易日开始
- `on_session_end(context, curTDate)`: 交易日结束
- `on_calculate(context)`: 策略计算（主K线闭合时触发）
- `on_bar(context, stdCode, period, newBar)`: K线闭合回调
- `on_tick(context, stdCode, newTick)`: Tick数据回调
- `on_backtest_end(context)`: 回测结束回调

#### BaseHftStrategy - HFT策略基类

**回调函数**：
- `on_init(context)`: 策略初始化
- `on_tick(context, stdCode, newTick)`: Tick数据回调
- `on_order_queue(context, stdCode, newOrdQue)`: 委托队列回调
- `on_order_detail(context, stdCode, newOrdDtl)`: 逐笔委托回调
- `on_transaction(context, stdCode, newTrans)`: 逐笔成交回调
- `on_entrust(context, stdCode, newEntrust)`: 委托回报回调
- `on_order(context, stdCode, newOrdInfo)`: 订单回报回调
- `on_trade(context, stdCode, newTrdInfo)`: 成交回报回调

#### BaseSelStrategy - 选股策略基类

**回调函数**：
- `on_init(context)`: 策略初始化
- `on_schedule(context, curDate, curTime)`: 定时调度回调
- `on_backtest_end(context)`: 回测结束回调

### 1.2.5 管理器模块

#### ProductMgr.py - 品种管理器

**功能**：管理品种信息（合约乘数、最小变动价位、保证金率等）

**核心方法**：
- `addProduct()`: 添加品种信息
- `getProduct()`: 获取品种信息
- `getProductInfo()`: 获取品种详细信息

#### ContractMgr.py - 合约管理器

**功能**：管理合约信息（合约代码、交易所、上市日期等）

**核心方法**：
- `loadContracts()`: 加载合约列表
- `getContract()`: 获取合约信息
- `getTotalCodes()`: 获取所有合约代码

#### SessionMgr.py - 交易时段管理器

**功能**：管理交易时段模板（集合竞价、连续交易、夜盘等）

**核心方法**：
- `loadSessions()`: 加载时段模板
- `getSession()`: 获取时段信息
- `sessionBegin()`: 时段开始检查
- `sessionEnd()`: 时段结束检查

### 1.2.6 monitor 子模块 - 监控与调度服务

#### WtMonSvr.py - 监控服务核心

**功能**：
- 提供 HTTP RESTful API 接口
- 实时监控策略运行状态
- 查看策略资金、持仓、订单等数据
- 组合盘管理（启动、停止、重启）

**API接口示例**：
- `GET /qrygrp`: 查询所有组合盘
- `GET /qrygrp/:grpid`: 查询指定组合盘信息
- `GET /qrygrpfunds/:grpid`: 查询组合盘资金数据
- `GET /qrygrppositions/:grpid`: 查询组合盘持仓数据
- `POST /startgrp/:grpid`: 启动组合盘
- `POST /stopgrp/:grpid`: 停止组合盘

#### PushSvr.py - WebSocket推送服务

**功能**：
- 实时推送策略运行事件
- 推送成交、订单、资金变化等实时数据
- 支持多客户端连接

#### WatchDog.py - 进程守护服务

**功能**：
- 监控进程运行状态
- 自动重启异常进程
- 定时任务调度（支持按周重复）
- 进程内存监控

**核心功能**：
- 进程健康检查
- 异常自动重启
- 调度任务管理
- 资源使用监控

#### WtBtMon.py - 回测任务管理器

**功能**：
- 管理回测任务的生命周期
- 接收回测进度和结果通知
- 支持回测任务队列管理

#### EventReceiver.py - 事件接收器

**功能**：
- 通过消息队列接收组合盘推送的事件
- 接收成交、订单、通知、日志等事件
- 处理后转发给监控服务

**支持的事件类型**：
- `TRD_TRADE`: 成交通知
- `TRD_ORDER`: 订单通知
- `TRD_NOTIFY`: 普通通知
- `LOG`: 日志通知

### 1.2.7 apps 子模块 - 应用层工具

#### WtBtAnalyst.py - 回测分析器

**功能**：
- 计算回测指标（年化收益、最大回撤、Sharpe比率、胜率等）
- 生成回测报告（Excel格式）
- 资金曲线分析
- 交易统计（交易次数、平均持仓时间等）

**核心指标**：
- 收益指标：总收益率、年化收益率、累计净值
- 风险指标：最大回撤、波动率、下行波动率
- 风险调整收益：Sharpe比率、Sortino比率、Calmar比率
- 交易统计：胜率、盈亏比、平均持仓时间

#### WtCtaOptimizer.py - CTA策略优化器

**功能**：
- 多进程并行回测
- 参数网格搜索
- 多目标优化（收益、回撤、Sharpe等）
- 优化结果汇总输出（CSV格式）

**使用示例**：
```python
optimizer = WtCtaOptimizer()
optimizer.add_strategy(...)
optimizer.set_gencfg(...)
optimizer.optimize(...)
```

#### WtCtaGAOptimizer.py - 遗传算法优化器

**功能**：
- 基于遗传算法的参数优化
- 适应度函数自定义
- 支持大规模参数空间搜索

#### WtHotPicker.py - 主力合约选择器

**功能**：
- 确定期货主力合约和次主力合约
- 支持从交易所网站爬取换月规则
- 支持解析 datakit 生成的 snapshot.csv

**数据源**：
- 交易所网站爬取
- datakit snapshot.csv 解析
- 人工配置规则

#### datahelper 子模块 - 数据辅助工具

**功能**：将不同数据源的数据转换为 WonderTrader 标准格式

**支持的数据源**：
- `DHRqData.py`: 米筐数据
- `DHTushare.py`: Tushare数据
- `DHTqSdk.py`: 天勤数据
- `DHBaostock.py`: 聚宽数据

**核心接口**：
- `dumpBars()`: 导出K线数据
- `dumpTicks()`: 导出Tick数据

### 1.2.8 数据定义模块

#### WtDataDefs.py - 数据容器定义

**功能**：定义高效的数据容器，基于 numpy 数组

**核心类**：

**WtNpKline** - K线数据容器
- 基于 numpy ndarray
- 支持高效的数据访问和切片
- 字段：time, open, high, low, close, volume, turnover, open_interest

**WtNpTicks** - Tick数据容器
- 支持批量Tick数据
- 高效的时间序列查询
- 字段：time, price, volume, turnover, open_interest, bid/ask prices

**WtNpOrdQueues** - 委托队列容器
- 逐笔委托队列数据
- 字段：time, price, orders, items

**WtNpOrdDetails** - 逐笔委托容器
- 逐笔委托明细数据
- 字段：time, index, side, price, qty, ord_no

**WtNpTransactions** - 逐笔成交容器
- 逐笔成交明细数据
- 字段：time, index, side, price, qty, trd_no

#### WtCoreDefs.py - 核心数据结构定义

**功能**：定义与 C++ 层交互的数据结构

**核心结构**：
- `WTSTickStruct`: Tick数据结构（C结构体）
- `WTSBarStruct`: K线数据结构（C结构体）
- `WTSOrdQueStruct`: 委托队列结构
- `WTSOrdDtlStruct`: 逐笔委托结构
- `WTSTransStruct`: 逐笔成交结构

**回调函数类型定义**：
- `CB_STRATEGY_INIT`: 策略初始化回调
- `CB_STRATEGY_TICK`: Tick数据回调
- `CB_STRATEGY_CALC`: 策略计算回调
- `CB_STRATEGY_BAR`: K线闭合回调

### 1.2.9 扩展模块接口

#### ExtModuleDefs.py - 扩展模块基类定义

**功能**：定义扩展模块的基础接口

**扩展模块类型**：

**BaseExtParser** - 扩展行情解析器
- `on_tick()`: Tick数据处理
- `on_bar()`: K线数据处理

**BaseExtExecuter** - 扩展执行器  
- `on_init()`: 执行器初始化
- `on_cmd()`: 执行命令处理

**BaseExtDataLoader** - 扩展数据加载器
- `load_bar()`: 加载K线数据
- `load_tick()`: 加载Tick数据

#### ExtToolDefs.py - 扩展工具基类定义

**功能**：定义扩展工具的基础接口

**工具类型**：

**BaseIndexWriter** - 指标输出器
- `on_calc()`: 指标计算回调
- `write()`: 写入指标数据

**BaseDataReporter** - 数据报告器
- `on_rt_data()`: 实时数据报告
- `on_settle()`: 结算数据报告

### 1.2.10 辅助工具模块

#### CodeHelper.py - 代码辅助工具

**功能**：提供合约代码转换、标准化等工具方法

**核心方法**：
- `stdCodeToStdCode()`: 代码标准化转换
- `splitStdCode()`: 解析标准化代码
- `getCodeStdCode()`: 获取标准代码

#### WtUtilDefs.py - 工具定义

**功能**：定义通用工具函数和装饰器

**核心工具**：
- `singleton`: 单例模式装饰器
- `PlatformHelper`: 平台辅助工具（获取动态库路径等）

---

## 1.3 数据流转机制

### 1.3.1 实盘数据流

```
行情源 (CTP/XTP/OES等)
    ↓
Parser (行情解析器)
    ↓
WtDtCore (数据核心)
    ↓
    ├─→ 实时数据广播 (UDP)
    ├─→ 历史数据落地 (文件存储)
    └─→ 策略引擎 (WtCore)
            ↓
        WtWrapper (Python包装)
            ↓
        CtaContext/HftContext
            ↓
        策略代码 (Python)
```

### 1.3.2 回测数据流

```
历史数据文件 (CSV/BIN格式)
    ↓
WtBtCore (回测核心)
    ↓
HisDataReplayer (数据回放器)
    ↓
MatchEngine (撮合引擎)
    ↓
WtBtWrapper (Python包装)
    ↓
CtaContext/HftContext  
    ↓
策略代码 (Python)
    ↓
回测结果输出
```

### 1.3.3 交易信号流

```
策略代码 (Python)
    ↓
Context.stra_enter_long/short()
    ↓
WtWrapper/WtBtWrapper
    ↓
C++ 引擎 (WtCore/WtBtCore)
    ↓
执行器 (WtExeFact)
    ↓
交易接口 (TraderCTP/TraderXTP等)
    ↓
交易所
```

---

## 1.4 配置体系

### 1.4.1 配置文件结构

**主配置文件** (config.yaml / config.json):
```yaml
basefiles:
  commodity: "common/commodities.json"
  contract: "common/contracts.json"
  holiday: "common/holidays.json"
  session: "common/sessions.json"

broadcaster:
  active: true
  bport: 3998
  sport: 3999

executers:
  - name: executer1
    module: WtLocalExecuter
    ...

strategies:
  - name: Strategy1
    active: true
    ...
```

### 1.4.2 基础数据文件

- **commodities.json**: 品种信息（合约乘数、最小变动价位等）
- **contracts.json**: 合约信息（合约代码、交易所、上市日期等）
- **sessions.json**: 交易时段模板
- **holidays.json**: 节假日配置
- **hots.json**: 主力合约换月规则

## 1.5 性能特性

### 1.5.1 数据访问性能

- **WtNpKline/WtNpTicks**: 基于 numpy 数组，数据访问性能提升约 10 倍
- **内存映射**: 历史数据使用 mmap，减少内存占用
- **数据缓存**: 策略上下文自动缓存常用数据

### 1.5.2 策略执行性能

- **CTA引擎**: Python策略单次重算约70微秒，C++策略约4.5微秒
- **HFT引擎**: 系统延迟1-2微秒
- **UFT引擎**: 系统延迟200纳秒以内

### 1.5.3 并发支持

- **多策略并发**: 支持同时运行多个策略
- **多进程回测**: WtCtaOptimizer 支持多进程并行回测
- **线程池**: CTA引擎支持线程池并发回调

---

## 1.6 总结

wtpy 框架特点：

1. **分层清晰**: 应用层、封装层、核心层职责明确
2. **接口统一**: 三种策略类型（CTA/HFT/SEL）使用统一的接口规范
3. **扩展性强**: 支持扩展解析器、执行器、数据加载器等
4. **性能优化**: 基于 numpy 的高效数据容器，C++ 核心高性能保证
5. **工具完善**: 提供回测分析、参数优化、监控服务等完整工具链
6. **易于使用**: Python 友好的接口，丰富的示例代码

# 第二部分：WonderTrader 与 wtpy 协作机制

## 2.1 整体协作架构

WonderTrader 与 wtpy 采用**分层协作架构**，C++ 核心负责高性能底层功能，Python 层负责易用性和应用层功能。

```
┌─────────────────────────────────────────────────────────────┐
│                    Python 应用层 (wtpy)                      │
│  ┌────────────┐  ┌────────────┐  ┌────────────┐          │
│  │ 策略开发    │  │ 监控服务    │  │ 回测分析    │          │
│  │ (Python)   │  │ (WtMonSvr)  │  │ (WtBtAnalyst)│          │
│  └────────────┘  └────────────┘  └────────────┘          │
└─────────────────────────────────────────────────────────────┘
                        ↕ Python API
┌─────────────────────────────────────────────────────────────┐
│                  Python 封装层 (wtpy wrapper)               │
│  ┌────────────┐  ┌────────────┐  ┌────────────┐          │
│  │ WtWrapper  │  │WtBtWrapper │  │WtDtWrapper │          │
│  │ (ctypes)   │  │ (ctypes)    │  │ (ctypes)   │          │
│  └────────────┘  └────────────┘  └────────────┘          │
└─────────────────────────────────────────────────────────────┘
                        ↕ C Interface (FFI)
┌─────────────────────────────────────────────────────────────┐
│              C++ 核心层 (WonderTrader)                       │
│  ┌────────────┐  ┌────────────┐  ┌────────────┐          │
│  │ WtPorter    │  │WtBtPorter  │  │WtDtPorter  │          │
│  │ (C接口导出) │  │ (C接口导出)│  │ (C接口导出)│          │
│  └────────────┘  └────────────┘  └────────────┘          │
│  ┌──────────────────────────────────────────────────────┐    │
│  │  WtCore / WtBtCore / WtDtCore (C++核心引擎)        │    │
│  └──────────────────────────────────────────────────────┘    │
└─────────────────────────────────────────────────────────────┘
```

## 2.2 接口对接机制

### 2.2.1 C 接口导出层 (WonderTrader 端)

**WtPorter.cpp** - 实盘引擎 C 接口导出

**核心功能**：
- 导出 C 风格函数接口（extern "C"）
- 提供策略注册、数据查询、交易指令等接口
- 管理 Python 回调函数的注册和调用

**典型接口**：
```cpp
// 初始化引擎
extern "C" bool init_engine(const char* cfgfile, bool isFile);

// 注册CTA策略回调
extern "C" void register_cta_strategy(unsigned int id, 
    CB_STRATEGY_INIT init_cb,
    CB_STRATEGY_TICK tick_cb,
    CB_STRATEGY_CALC calc_cb,
    ...);

// 策略下单接口
extern "C" bool cta_enter_long(unsigned int id, const char* code, 
    double qty, const char* tag, double price, double stop);

// 获取K线数据
extern "C" bool cta_get_bars(unsigned int id, const char* code, 
    const char* period, int count, WTSBarStruct* bars);
```

**WtBtPorter.cpp** - 回测引擎 C 接口导出

**核心功能**：
- 导出回测相关的 C 接口
- 支持回测控制（单步执行、时间设置等）
- 提供回测结果数据查询接口

**WtDtPorter.cpp** - 数据组件 C 接口导出

**核心功能**：
- 导出数据组件相关接口
- 支持历史数据加载
- 支持扩展解析器注册

### 2.2.2 Python 封装层 (wtpy wrapper)

**WtWrapper.py** - 实盘引擎 Python 封装

**工作机制**：

1. **动态库加载**：
```python
from ctypes import cdll, c_char_p, c_bool, c_ulong, c_double, POINTER
import os

# 获取动态库路径
dllname = ph.getModule("WtPorter")  # Windows: "WtPorter.dll", Linux: "libWtPorter.so"
_path = os.path.join(wrapper_dir, dllname)

# 加载动态库
self.api = cdll.LoadLibrary(_path)
```

2. **函数签名定义**：
```python
# 设置函数返回类型
self.api.get_version.restype = c_char_p

# 设置函数参数类型
self.api.cta_enter_long.argtypes = [
    c_ulong,      # strategy_id
    c_char_p,      # code
    c_double,      # qty
    c_char_p,      # tag
    c_double,      # price
    c_double       # stop
]
self.api.cta_enter_long.restype = c_bool
```

3. **Python 接口封装**：
```python
def enter_long(self, strategy_id: int, code: str, qty: float, 
                tag: str = "", price: float = 0.0, stop: float = 0.0) -> bool:
    """开多仓接口"""
    return self.api.cta_enter_long(
        c_ulong(strategy_id),
        code.encode('utf-8'),
        c_double(qty),
        tag.encode('utf-8'),
        c_double(price),
        c_double(stop)
    )
```

4. **回调函数注册**：
```python
# 定义回调函数类型
CB_STRATEGY_INIT = CFUNCTYPE(None, c_ulong)
CB_STRATEGY_TICK = CFUNCTYPE(None, c_ulong, c_char_p, POINTER(WTSTickStruct))

# 创建Python回调函数
def _on_init_cb(strategy_id):
    ctx = self._engine.get_context(strategy_id)
    strategy = ctx._stra_info__
    strategy.on_init(ctx)

# 注册回调
self.api.register_cta_strategy(
    strategy_id,
    CB_STRATEGY_INIT(_on_init_cb),
    CB_STRATEGY_TICK(_on_tick_cb),
    ...
)
```

**数据转换机制**：

**C结构体 ↔ Python对象**：
```python
# C结构体定义
class WTSTickStruct(Structure):
    _fields_ = [
        ("time", c_uint64),
        ("price", c_double),
        ("volume", c_double),
        ...
    ]

# Python中创建结构体
tick_ptr = POINTER(WTSTickStruct)()
self.api.cta_get_last_tick(strategy_id, code.encode(), tick_ptr)

# 转换为Python字典
tick_dict = {
    "time": tick_ptr.contents.time,
    "price": tick_ptr.contents.price,
    "volume": tick_ptr.contents.volume,
    ...
}
```

**NumPy数组 ↔ C数组**：
```python
# 获取K线数据（返回NumPy数组）
bars = np.zeros((count,), dtype=WTSBarStruct)
self.api.cta_get_bars(
    strategy_id,
    code.encode(),
    period.encode(),
    count,
    bars.ctypes.data_as(POINTER(WTSBarStruct))
)

# 转换为Python友好格式
kline_data = WtNpKline(bars)
```

### 2.2.3 Python 调用 C++ 动态库的完整机制详解

本节详细解释 Python 如何通过 ctypes 调用 C++ 编译的动态库，这是 wtpy 与 WonderTrader 协作的核心技术基础。

**核心机制概述**：

Python 通过 `ctypes` 标准库调用 C++ 动态库，整个过程包括：
1. **加载动态库**：使用 `cdll.LoadLibrary()` 将 `.dll`/`.so` 文件加载到内存
2. **定义函数签名**：通过 `argtypes` 和 `restype` 指定参数和返回值类型
3. **定义数据结构**：使用 `Structure` 定义与 C++ 对应的结构体
4. **注册回调函数**：使用 `CFUNCTYPE` 创建函数指针，实现 C++ → Python 通信
5. **调用 C++ 函数**：将 Python 对象转换为 C 类型，调用 C++ 函数

**完整调用流程**：
```
Python 代码
    ↓
ctypes 库（Python 标准库）
    ↓
C++ 动态库（WtPorter.dll / libWtPorter.so）
    ↓
WonderTrader C++ 核心引擎
```

**关键技术点**：
- **动态库加载**：操作系统负责查找和加载动态库及其依赖
- **类型映射**：Python 类型与 C 类型之间的自动/手动转换
- **内存管理**：结构体指针传递，避免数据拷贝
- **回调机制**：C++ 通过函数指针调用 Python 函数
- **字符串编码**：Windows 使用 GBK，Linux 使用 UTF-8

#### 2.2.3.1 动态库加载机制

**第一步：确定动态库路径**

Python 需要知道动态库文件的位置。wtpy 使用 `PlatformHelper` 来根据平台自动生成正确的路径：

```python
# PlatformHelper.py
class PlatformHelper:
    @staticmethod
    def getModule(moduleName: str) -> str:
        """根据平台生成动态库文件名"""
        if PlatformHelper.isWindows():
            # Windows: x64/WtPorter.dll 或 x86/WtPorter.dll
            if PlatformHelper.isPythonX64():
                return "x64/WtPorter.dll"
            else:
                return "x86/WtPorter.dll"
        else:
            # Linux: linux/libWtPorter.so
            return "linux/libWtPorter.so"
```

**第二步：加载动态库**

使用 Python 标准库 `ctypes` 的 `cdll.LoadLibrary()` 加载动态库：

```python
# WtWrapper.py
from ctypes import cdll
import os

# 获取动态库路径
dllname = ph.getModule("WtPorter")  # 例如: "x64/WtPorter.dll"
_path = os.path.join(wrapper_dir, dllname)

# 加载动态库到内存
self.api = cdll.LoadLibrary(_path)
```

**加载过程说明**：

1. **操作系统查找动态库**：
   - Windows: 在当前目录、系统目录、PATH 环境变量中查找 `.dll` 文件
   - Linux: 在当前目录、`LD_LIBRARY_PATH` 环境变量、系统库目录中查找 `.so` 文件

2. **加载依赖库**：
   - 如果 `WtPorter.dll` 依赖其他 DLL（如 `WtCore.dll`），操作系统会自动加载这些依赖库

3. **符号解析**：
   - 操作系统解析动态库的导出表，找到所有标记为 `EXPORT_FLAG` 的函数
   - 这些函数名会被添加到符号表中，供 Python 调用

**加载后的对象**：
- `self.api` 是一个 `ctypes.CDLL` 对象
- 它包含了动态库中所有导出函数的引用
- 可以通过 `self.api.函数名` 的方式访问这些函数

#### 2.2.3.2 函数签名定义机制

**为什么需要定义函数签名？**

C++ 函数调用需要：
1. **参数类型匹配**：确保 Python 对象正确转换为 C 类型
2. **参数顺序正确**：按照 C 函数定义的顺序传递参数
3. **返回值类型正确**：确保 C 返回值正确转换为 Python 对象
4. **调用约定正确**：确保函数调用栈的正确管理

**第一步：定义 C 数据类型映射**

Python 的 `ctypes` 提供了与 C 类型对应的 Python 类型：

```python
from ctypes import (
    c_int32,      # C: int32_t
    c_uint32,     # C: unsigned int
    c_uint64,     # C: unsigned long long
    c_double,     # C: double
    c_char_p,     # C: char* (字符串指针)
    c_bool,       # C: bool
    c_ulong,      # C: unsigned long
    POINTER,      # C: 指针类型
    Structure     # C: 结构体
)
```

**类型映射表**：

| C++ 类型 | ctypes 类型 | Python 类型 | 说明 |
|---------|------------|------------|------|
| `int` | `c_int32` | `int` | 32位有符号整数 |
| `unsigned int` | `c_uint32` | `int` | 32位无符号整数 |
| `unsigned long long` | `c_uint64` | `int` | 64位无符号整数 |
| `double` | `c_double` | `float` | 双精度浮点数 |
| `bool` | `c_bool` | `bool` | 布尔值 |
| `char*` | `c_char_p` | `bytes` | 字符串指针 |
| `const char*` | `c_char_p` | `bytes` | 常量字符串指针 |
| `void*` | `c_void_p` | `int` | 通用指针 |
| `T*` | `POINTER(T)` | - | 指针类型 |

**第二步：设置函数参数类型**

使用 `argtypes` 属性设置函数参数类型列表：

```python
# C++ 函数签名：
# bool cta_enter_long(unsigned long id, const char* code, double qty, 
#                     const char* tag, double price, double stop);

# Python 中设置参数类型
self.api.cta_enter_long.argtypes = [
    c_ulong,      # id: 策略ID
    c_char_p,     # code: 合约代码（字符串）
    c_double,     # qty: 数量
    c_char_p,     # tag: 用户标记（字符串）
    c_double,     # price: 限价
    c_double      # stop: 止损价
]
```

**第三步：设置函数返回类型**

使用 `restype` 属性设置函数返回类型：

```python
# C++ 函数签名：
# const char* get_version();

# Python 中设置返回类型
self.api.get_version.restype = c_char_p  # 返回字符串指针
```

**完整示例**：

```python
# WtWrapper.py 中的初始化代码
def __init__(self, engine):
    # 加载动态库
    self.api = cdll.LoadLibrary(_path)
    
    # 设置 get_version 的返回类型
    self.api.get_version.restype = c_char_p
    
    # 设置 cta_enter_long 的参数和返回类型
    self.api.cta_enter_long.argtypes = [
        c_ulong, c_char_p, c_double, c_char_p, c_double, c_double
    ]
    self.api.cta_enter_long.restype = c_bool
    
    # 设置 cta_get_position 的返回类型
    self.api.cta_get_position.restype = c_double
    
    # ... 更多函数签名设置
```

#### 2.2.3.3 函数调用机制

**第一步：参数转换**

Python 调用 C++ 函数时，需要将 Python 对象转换为 C 类型：

```python
def cta_enter_long(self, strategy_id: int, code: str, qty: float, 
                    tag: str = "", price: float = 0.0, stop: float = 0.0) -> bool:
    """开多仓接口"""
    
    # Python 字符串 → C char* (需要编码)
    code_bytes = code.encode('utf-8')  # Python str → bytes
    tag_bytes = tag.encode('utf-8')
    
    # Python int → C unsigned long
    id_c = c_ulong(strategy_id)
    
    # Python float → C double
    qty_c = c_double(qty)
    price_c = c_double(price)
    stop_c = c_double(stop)
    
    # 调用 C++ 函数
    result = self.api.cta_enter_long(
        id_c,           # c_ulong
        code_bytes,     # c_char_p (bytes)
        qty_c,          # c_double
        tag_bytes,      # c_char_p (bytes)
        price_c,        # c_double
        stop_c          # c_double
    )
    
    # C bool → Python bool
    return bool(result)
```

**参数转换规则**：

1. **整数类型**：
   - Python `int` → `c_ulong(value)` 或 `c_uint32(value)`
   - ctypes 会自动转换，但显式转换更安全

2. **浮点数类型**：
   - Python `float` → `c_double(value)`
   - ctypes 会自动转换

3. **字符串类型**：
   - Python `str` → `bytes` → `c_char_p`
   - **重要**：必须先用 `.encode()` 转换为 bytes
   - Windows 通常使用 GBK 编码，Linux 使用 UTF-8

4. **布尔类型**：
   - Python `bool` → `c_bool(value)`
   - ctypes 会自动转换

**第二步：函数调用**

调用 C++ 函数时，ctypes 会：
1. 检查参数类型是否匹配 `argtypes`
2. 将 Python 对象转换为 C 类型
3. 将参数压入调用栈（按照 C 调用约定）
4. 跳转到 C++ 函数地址执行
5. 获取返回值并转换为 Python 对象

**调用示例**：

```python
# 简单调用（ctypes 自动转换）
version = self.api.get_version()  # 返回 bytes，需要解码
version_str = version.decode('utf-8')

# 复杂调用（需要显式转换）
result = self.api.cta_enter_long(
    c_ulong(12345),              # 策略ID
    b"SHFE.rb.2305",             # 合约代码（bytes）
    c_double(10.0),              # 数量
    b"my_tag",                   # 用户标记（bytes）
    c_double(3500.0),            # 限价
    c_double(3450.0)              # 止损价
)
```

#### 2.2.3.4 数据结构转换机制

**C 结构体 ↔ Python 对象**

**第一步：定义 C 结构体**

使用 `ctypes.Structure` 定义与 C++ 结构体对应的 Python 类：

```python
# WtCoreDefs.py
from ctypes import Structure, c_char, c_uint32, c_uint64, c_double

# C++ 结构体定义（对应）：
# struct WTSTickStruct {
#     char exchg[16];
#     char code[32];
#     double price;
#     uint32_t action_date;
#     uint32_t action_time;
#     ...
# };

class WTSTickStruct(Structure):
    """Tick数据结构，与C++中的WTSTickStruct完全对应"""
    
    # 定义字段列表，必须与C++结构体字段顺序和类型完全一致
    _fields_ = [
        ("exchg", c_char * 16),      # 交易所代码，16个字符的数组
        ("code", c_char * 32),       # 合约代码，32个字符的数组
        ("price", c_double),          # 最新价
        ("open", c_double),           # 开盘价
        ("high", c_double),           # 最高价
        ("low", c_double),            # 最低价
        ("settle_price", c_double),   # 结算价
        ("upper_limit", c_double),   # 涨停价
        ("lower_limit", c_double),   # 跌停价
        ("total_volume", c_double),   # 总成交量
        ("volume", c_double),         # 本笔成交量
        ("trading_date", c_uint32),   # 交易日
        ("action_date", c_uint32),    # 动作日期
        ("action_time", c_uint32),    # 动作时间
        # ... 更多字段（买卖盘10档等）
    ]
    
    # 结构体对齐方式（可选，通常8字节对齐）
    _pack_ = 8
```

**第二步：接收 C++ 传递的结构体指针**

C++ 函数通常通过指针传递结构体：

```python
# C++ 函数签名：
# void on_stra_tick(unsigned long id, const char* code, WTSTickStruct* tick);

# Python 回调函数定义
def on_stra_tick(self, id: int, code: bytes, tick_ptr: POINTER(WTSTickStruct)):
    """Tick数据回调"""
    
    # 获取结构体内容
    tick = tick_ptr.contents  # 通过 .contents 访问指针指向的结构体
    
    # 访问结构体字段
    price = tick.price         # double → Python float
    volume = tick.volume        # double → Python float
    
    # 字符数组 → Python 字符串
    code_str = tick.code.decode('utf-8').rstrip('\x00')  # 去除末尾的 \0
    exchg_str = tick.exchg.decode('utf-8').rstrip('\x00')
    
    # 整数类型自动转换
    action_date = tick.action_date  # uint32 → Python int
    action_time = tick.action_time  # uint32 → Python int
    
    # 转换为Python字典（可选）
    tick_dict = {
        "code": code_str,
        "exchg": exchg_str,
        "price": price,
        "volume": volume,
        "action_date": action_date,
        "action_time": action_time,
        # ... 更多字段
    }
    
    return tick_dict
```

**第三步：向 C++ 传递结构体**

如果需要向 C++ 传递结构体，需要创建结构体实例：

```python
# 创建结构体实例
tick = WTSTickStruct()

# 设置字段值
tick.price = 3500.0
tick.volume = 100.0
tick.code = b"SHFE.rb.2305"  # 字符数组需要用 bytes
tick.action_date = 20230220
tick.action_time = 93000000  # 09:30:00.000

# 传递结构体指针给 C++ 函数
tick_ptr = POINTER(WTSTickStruct)(tick)  # 创建指针
self.api.some_function(tick_ptr)
```

**NumPy 数组 ↔ C 数组**

对于数组数据，wtpy 使用 NumPy 数组来高效处理：

```python
# WtDataDefs.py
import numpy as np
from ctypes import POINTER, addressof

# 定义 NumPy 数据类型（对应 C 结构体）
NpTypeBar = np.dtype([
    ('date', 'u4'),      # uint32
    ('time', 'u8'),      # uint64
    ('open', 'd'),       # double
    ('high', 'd'),
    ('low', 'd'),
    ('close', 'd'),
    ('volume', 'd'),
    # ...
])

class WtNpKline:
    """基于NumPy的K线数据容器"""
    
    def set_data(self, bar_ptr: POINTER(WTSBarStruct), count: int):
        """从C结构体数组设置数据"""
        # 获取C数组的起始地址
        addr = addressof(bar_ptr.contents)
        
        # 将C数组转换为NumPy数组（零拷贝）
        self.__data__ = np.frombuffer(
            (WTSBarStruct * count).from_address(addr),
            dtype=NpTypeBar
        )
```

**数据转换流程**：

```
C++ 端：
  WTSBarStruct bars[100];  // C数组
  ↓
  Python 回调函数接收：
  POINTER(WTSBarStruct) bar_ptr  // C指针
  ↓
  NumPy 转换：
  np.frombuffer(...)  // 零拷贝转换为NumPy数组
  ↓
  Python 使用：
  WtNpKline 对象，提供便捷的访问接口
```

#### 2.2.3.5 回调函数注册机制

**为什么需要回调函数？**

C++ 引擎需要通知 Python 策略各种事件（行情到达、K线闭合、订单回报等）。回调函数实现了 C++ → Python 的通信。

**第一步：定义回调函数类型**

使用 `ctypes.CFUNCTYPE` 定义回调函数类型：

```python
# WtCoreDefs.py
from ctypes import CFUNCTYPE, c_void_p, c_ulong, c_char_p, POINTER

# C++ 回调函数类型：
# typedef void (*FuncStraInitCallback)(unsigned long id);

# Python 中定义对应的回调函数类型
CB_STRATEGY_INIT = CFUNCTYPE(
    c_void_p,      # 返回类型：void
    c_ulong        # 参数类型：策略ID
)

# 更复杂的回调函数类型
CB_STRATEGY_TICK = CFUNCTYPE(
    c_void_p,                      # 返回类型：void
    c_ulong,                       # 参数1：策略ID
    c_char_p,                      # 参数2：合约代码
    POINTER(WTSTickStruct)         # 参数3：Tick数据指针
)
```

**CFUNCTYPE 语法**：
```python
CFUNCTYPE(返回类型, 参数类型1, 参数类型2, ...)
```

**第二步：创建 Python 回调函数**

编写实际的 Python 回调函数：

```python
# WtWrapper.py
def on_stra_init(self, id: int):
    """策略初始化回调函数"""
    # 获取策略上下文
    engine = self._engine
    ctx = engine.get_context(id)
    
    if ctx is not None:
        # 调用策略的初始化方法
        ctx.on_init()
    return

def on_stra_tick(self, id: int, code: bytes, tick_ptr: POINTER(WTSTickStruct)):
    """Tick数据回调函数"""
    # 获取策略上下文
    engine = self._engine
    ctx = engine.get_context(id)
    
    if ctx is not None:
        # 解码合约代码
        code_str = code.decode('utf-8')
        
        # 调用策略的Tick回调
        ctx.on_tick(code_str, tick_ptr)
    return
```

**第三步：将 Python 函数包装为 C 回调**

使用 `CFUNCTYPE` 将 Python 函数包装为 C 回调函数：

```python
# 创建C回调函数对象
self.cb_stra_init = CB_STRATEGY_INIT(self.on_stra_init)
self.cb_stra_tick = CB_STRATEGY_TICK(self.on_stra_tick)
```

**第四步：注册回调到 C++**

将回调函数注册到 C++ 引擎：

```python
# C++ 函数签名：
# void register_cta_callbacks(
#     FuncStraInitCallback cbInit,
#     FuncStraTickCallback cbTick,
#     ...
# );

# Python 中注册回调
self.api.register_cta_callbacks(
    self.cb_stra_init,      # 初始化回调
    self.cb_stra_tick,      # Tick回调
    self.cb_stra_calc,      # 计算回调
    self.cb_stra_bar,       # K线回调
    self.cb_session_event,  # 会话事件回调
    self.cb_stra_cond_trigger  # 条件单回调
)
```

**回调函数调用流程**：

```
1. C++ 引擎事件发生（如行情到达）
   ↓
2. C++ 调用注册的回调函数
   register_cta_callbacks(...) 中注册的 cb_stra_tick
   ↓
3. ctypes 将C参数转换为Python对象
   c_ulong → Python int
   c_char_p → Python bytes
   POINTER(WTSTickStruct) → Python POINTER对象
   ↓
4. 调用Python回调函数
   self.on_stra_tick(id, code, tick_ptr)
   ↓
5. Python处理事件
   ctx.on_tick(code_str, tick_ptr)
   ↓
6. 返回C++（如果需要）
   return None (对应C++的void)
```

**重要注意事项**：

1. **回调函数必须保持引用**：
   ```python
   # 正确：保存为实例变量
   self.cb_stra_init = CB_STRATEGY_INIT(self.on_stra_init)
   
   # 错误：临时变量会被垃圾回收
   cb = CB_STRATEGY_INIT(self.on_stra_init)  # 不要这样做！
   ```

2. **回调函数不能抛出异常**：
   ```python
   def on_stra_tick(self, id, code, tick_ptr):
       try:
           # 处理逻辑
           ctx.on_tick(...)
       except Exception as e:
           # 必须捕获异常，否则会导致C++崩溃
           print(f"Error in callback: {e}")
   ```

3. **字符串编码问题**：
   ```python
   # Windows: C++使用GBK，Python需要转换
   if PlatformHelper.isWindows():
       code_bytes = code.encode('gbk')
   else:
       code_bytes = code.encode('utf-8')
   ```

#### 2.2.3.6 完整调用示例

**示例：策略开多仓的完整流程**

```python
# 1. 用户代码调用
ctx.enter_long("SHFE.rb.2305", 10, tag="my_order")

# 2. CtaContext.py 中的实现
def enter_long(self, stdCode: str, qty: float, 
               userTag: str = "", limitPrice: float = 0.0, 
               stopPrice: float = 0.0):
    """开多仓"""
    # 调用底层包装器
    self.__wrapper__.cta_enter_long(
        self.__id__,      # 策略ID
        stdCode,          # 合约代码
        qty,              # 数量
        userTag,          # 用户标记
        limitPrice,       # 限价
        stopPrice         # 止损价
    )

# 3. WtWrapper.py 中的实现
def cta_enter_long(self, id: int, code: str, qty: float,
                    tag: str = "", price: float = 0.0, stop: float = 0.0):
    """开多仓接口"""
    # 参数转换
    code_bytes = ph.auto_encode(code)  # 自动编码（Windows→GBK，Linux→UTF-8）
    tag_bytes = ph.auto_encode(tag)
    
    # 调用C++函数
    return self.api.cta_enter_long(
        c_ulong(id),
        code_bytes,
        c_double(qty),
        tag_bytes,
        c_double(price),
        c_double(stop)
    )

# 4. C++ 端执行（WtPorter.cpp）
void cta_enter_long(CtxHandler cHandle, const char* stdCode, 
                    double qty, const char* userTag, 
                    double limitprice, double stopprice)
{
    // 获取运行时运行器
    WtRtRunner& runner = getRunner();
    
    // 调用C++核心引擎
    runner.ctaEnterLong(cHandle, stdCode, qty, userTag, limitprice, stopprice);
}

# 5. C++ 核心引擎处理（WtCore）
# 执行开仓逻辑，更新持仓，发送订单等
```

**示例：Tick数据回调的完整流程**

```python
# 1. C++ 引擎收到行情数据
# WtCore 中调用：
runner.onTickData(tick_data);

# 2. WtRtRunner 调用注册的回调
void WtRtRunner::onTickData(WTSTickData* tick) {
    if (_cb_cta_tick != NULL) {
        // 转换为C结构体
        WTSTickStruct tickStruct;
        // ... 数据转换 ...
        
        // 调用Python回调
        _cb_cta_tick(strategy_id, code, &tickStruct);
    }
}

# 3. Python 回调函数被调用
def on_stra_tick(self, id: int, code: bytes, tick_ptr: POINTER(WTSTickStruct)):
    """Tick数据回调"""
    # 获取策略上下文
    ctx = self._engine.get_context(id)
    if ctx is not None:
        # 调用策略的Tick回调
        ctx.on_tick(code.decode('utf-8'), tick_ptr)

# 4. CtaContext 处理Tick数据
def on_tick(self, stdCode: str, newTick: POINTER(WTSTickStruct)):
    """Tick数据回调"""
    # 转换为Python对象
    tick = newTick.contents
    tick_dict = {
        "price": tick.price,
        "volume": tick.volume,
        # ...
    }
    
    # 调用策略的on_tick方法
    self.__stra_info__.on_tick(self, stdCode, tick_dict)

# 5. 用户策略代码处理
def on_tick(self, context, code, tick):
    """策略Tick回调"""
    price = tick["price"]
    # 策略逻辑...
```

**示例：最简单的使用流程**

以下是一个最简单的使用示例，展示从加载动态库到调用函数的完整过程：

```python
# 最简单的使用示例
from ctypes import cdll, c_char_p, c_ulong, c_double
import os

# 1. 加载动态库
dll_path = "x64/WtPorter.dll"  # Windows 64位
api = cdll.LoadLibrary(dll_path)

# 2. 设置函数签名
api.get_version.restype = c_char_p  # 返回字符串

api.cta_get_price.argtypes = [c_char_p]  # 参数：合约代码（字符串）
api.cta_get_price.restype = c_double     # 返回：价格（浮点数）

# 3. 调用函数
# 获取版本信息
version_bytes = api.get_version()
version_str = version_bytes.decode('utf-8')
print(f"版本: {version_str}")

# 获取价格
code_bytes = "SHFE.rb.2305".encode('utf-8')
price = api.cta_get_price(code_bytes)
print(f"价格: {price}")
```

这个简单示例展示了：
- 如何加载动态库
- 如何设置函数签名
- 如何调用函数并处理返回值
- 字符串编码的重要性

#### 2.2.3.7 常见问题和解决方案

**问题1：动态库加载失败**

**错误信息**：
```
OSError: [WinError 126] 找不到指定的模块
```

**原因**：
- 动态库文件不存在
- 依赖的DLL缺失
- 架构不匹配（32位/64位）

**解决方案**：
```python
# 检查动态库是否存在
import os
dll_path = "x64/WtPorter.dll"
if not os.path.exists(dll_path):
    print(f"动态库不存在: {dll_path}")

# 使用 Dependency Walker (Windows) 或 ldd (Linux) 检查依赖
# Windows: dumpbin /DEPENDENTS WtPorter.dll
# Linux: ldd libWtPorter.so
```

**问题2：函数调用参数类型错误**

**错误信息**：
```
ctypes.ArgumentError: argument 2: <class 'TypeError'>: wrong type
```

**原因**：
- 参数类型不匹配
- 字符串未编码为bytes

**解决方案**：
```python
# 确保字符串编码
code_bytes = code.encode('utf-8')  # 不要直接传str

# 确保类型正确
self.api.cta_enter_long(
    c_ulong(id),        # 显式转换
    code_bytes,         # bytes类型
    c_double(qty),      # 显式转换
    # ...
)
```

**问题3：回调函数未被调用**

**原因**：
- 回调函数被垃圾回收
- 回调函数注册失败
- C++端未触发事件

**解决方案**：
```python
# 确保回调函数保存在实例变量中
self.cb_stra_init = CB_STRATEGY_INIT(self.on_stra_init)  # 保存引用

# 检查注册是否成功
try:
    self.api.register_cta_callbacks(...)
    print("回调注册成功")
except Exception as e:
    print(f"回调注册失败: {e}")
```

**问题4：字符串编码问题**

**错误信息**：
- Windows: 中文乱码
- Linux: 编码错误

**解决方案**：
```python
# 使用 PlatformHelper 自动处理编码
code_bytes = ph.auto_encode(code)  # Windows→GBK，Linux→UTF-8

# 或者手动处理
if PlatformHelper.isWindows():
    code_bytes = code.encode('gbk')
else:
    code_bytes = code.encode('utf-8')
```

#### 2.2.3.8 性能优化建议

1. **减少Python-C++调用次数**：
   - 批量获取数据而不是逐条获取
   - 使用NumPy数组批量传递数据

2. **使用零拷贝技术**：
   - 使用 `np.frombuffer()` 直接映射C数组内存
   - 避免不必要的数据拷贝

3. **缓存回调函数**：
   - 将回调函数保存在实例变量中
   - 避免重复创建回调函数对象

4. **异步处理**：
   - 对于耗时操作，使用异步回调
   - 避免阻塞C++线程

**总结：Python调用C++动态库的关键要点**

1. **动态库加载**：
   - 使用 `cdll.LoadLibrary()` 加载动态库
   - 操作系统自动处理依赖库加载和符号解析
   - 加载后的对象包含所有导出函数的引用

2. **函数签名定义**：
   - 必须通过 `argtypes` 和 `restype` 定义函数签名
   - 确保 Python 对象正确转换为 C 类型
   - 字符串必须编码为 `bytes` 类型

3. **数据结构转换**：
   - 使用 `Structure` 定义与 C++ 对应的结构体
   - 字段顺序和类型必须完全一致
   - 通过 `.contents` 访问指针指向的结构体

4. **回调函数注册**：
   - 使用 `CFUNCTYPE` 创建函数指针
   - 必须保存回调函数引用，避免被垃圾回收
   - 回调函数不能抛出异常

5. **字符串编码**：
   - Windows 使用 GBK 编码，Linux 使用 UTF-8
   - 使用 `PlatformHelper.auto_encode()` 自动处理
   - 返回的字符串需要解码

6. **性能优化**：
   - 使用 NumPy 数组批量传递数据
   - 使用零拷贝技术减少内存拷贝
   - 减少 Python-C++ 调用次数

## 2.3 数据流转协作

### 2.3.1 行情数据流

**完整流程**：

```
1. 行情源 (CTP/XTP/OES等)
   ↓
2. Parser (C++解析器)
   ├─ 解析原始数据
   ├─ 转换为WTSTickData
   └─ 调用IParserSpi回调
   ↓
3. WtDtCore (C++数据核心)
   ├─ 数据落地存储
   ├─ 实时数据广播 (UDP)
   └─ 触发策略引擎
   ↓
4. WtCore (C++策略引擎)
   ├─ 调用策略回调
   └─ 通过WtPorter调用Python回调
   ↓
5. WtWrapper (Python包装层)
   ├─ 接收C++回调
   ├─ 转换数据结构
   └─ 调用Python策略
   ↓
6. CtaContext (Python策略上下文)
   ├─ 缓存数据
   └─ 提供查询接口
   ↓
7. 策略代码 (Python用户代码)
   ├─ on_tick()回调
   └─ 调用context接口下单
```

**关键代码示例**：

**C++端 (WtCore)**：
```cpp
// 行情数据到达时
void WtEngine::on_tick_data(WTSTickData* tick) {
    // 遍历所有策略
    for (auto& strategy : strategies) {
        // 调用策略的tick回调
        if (strategy.on_tick_cb) {
            // 转换为C结构体
            WTSTickStruct tickStruct;
            convert_tick_data(tick, &tickStruct);
            
            // 调用Python回调
            strategy.on_tick_cb(
                strategy.id,
                tick->code(),
                &tickStruct
            );
        }
    }
}
```

**Python端 (WtWrapper)**：
```python
def _on_tick_callback(strategy_id, code_bytes, tick_ptr):
    """C++回调函数"""
    code = code_bytes.decode('utf-8')
    
    # 转换C结构体为Python字典
    tick_dict = {
        "time": tick_ptr.contents.time,
        "price": tick_ptr.contents.price,
        "volume": tick_ptr.contents.volume,
        ...
    }
    
    # 获取策略上下文
    ctx = self._engine.get_context(strategy_id)
    
    # 调用策略的on_tick方法
    strategy = ctx._stra_info__
    strategy.on_tick(ctx, code, tick_dict)
```

### 2.3.2 交易指令流

**完整流程**：

```
1. 策略代码 (Python)
   context.stra_enter_long(code, qty)
   ↓
2. CtaContext (Python)
   stra_enter_long() → wrapper.enter_long()
   ↓
3. WtWrapper (Python)
   enter_long() → api.cta_enter_long()
   ↓
4. WtPorter (C接口)
   cta_enter_long() → 引擎处理
   ↓
5. WtCore (C++引擎)
   ├─ 风控检查
   ├─ 仓位计算
   └─ 执行器调用
   ↓
6. WtExeFact (C++执行器)
   ├─ 算法执行 (TWAP/VWAP等)
   └─ 订单拆分
   ↓
7. TraderAdapter (C++交易接口)
   ├─ 订单格式化
   └─ 发送到交易所
   ↓
8. 交易所
```

**关键代码示例**：

**Python端 (CtaContext)**：
```python
def stra_enter_long(self, stdCode: str, qty: float, 
                    usertag: str = "", price: float = 0.0, 
                    stop: float = 0.0):
    """开多仓接口"""
    return self.__wrapper__.cta_enter_long(
        self.__id__,
        stdCode,
        qty,
        usertag,
        price,
        stop
    )
```

**C++端 (WtCore)**：
```cpp
// C接口实现
extern "C" bool cta_enter_long(unsigned int id, const char* code, 
    double qty, const char* tag, double price, double stop) {
    
    // 获取策略上下文
    CtaStrategy* strategy = get_strategy(id);
    if (!strategy) return false;
    
    // 执行开多仓逻辑
    return strategy->enter_long(code, qty, tag, price, stop);
}

// C++引擎内部处理
bool CtaEngine::enter_long(const std::string& code, double qty, ...) {
    // 1. 风控检查
    if (!risk_check(id, code, qty)) return false;
    
    // 2. 计算目标仓位
    double target_pos = get_position(id, code) + qty;
    
    // 3. 发送到执行器
    executer->set_position(code, target_pos);
    
    return true;
}
```

### 2.3.3 历史数据查询流

**完整流程**：

```
1. 策略代码 (Python)
   bars = context.stra_get_bars(code, period, count)
   ↓
2. CtaContext (Python)
   stra_get_bars() → wrapper.get_bars()
   ↓
3. WtWrapper (Python)
   get_bars() → api.cta_get_bars()
   ↓
4. WtPorter (C接口)
   cta_get_bars() → 引擎查询
   ↓
5. WtCore (C++引擎)
   ├─ 查询内存缓存
   ├─ 查询历史数据文件
   └─ 返回数据
   ↓
6. WtWrapper (Python)
   ├─ 接收C数组
   └─ 转换为NumPy数组
   ↓
7. CtaContext (Python)
   ├─ 封装为WtNpKline
   └─ 返回给策略
```

**关键代码示例**：

**Python端 (WtWrapper)**：
```python
def get_bars(self, strategy_id: int, code: str, 
              period: str, count: int) -> WtNpKline:
    """获取K线数据"""
    # 创建C数组
    bars = (WTSBarStruct * count)()
    
    # 调用C接口
    ret = self.api.cta_get_bars(
        c_ulong(strategy_id),
        code.encode('utf-8'),
        period.encode('utf-8'),
        c_int(count),
        bars
    )
    
    if not ret:
        return None
    
    # 转换为NumPy数组
    bars_array = np.frombuffer(bars, dtype=WTSBarStruct)
    
    # 封装为WtNpKline
    return WtNpKline(bars_array)
```

**C++端 (WtCore)**：
```cpp
extern "C" bool cta_get_bars(unsigned int id, const char* code,
    const char* period, int count, WTSBarStruct* bars) {
    
    // 获取策略上下文
    CtaStrategy* strategy = get_strategy(id);
    if (!strategy) return false;
    
    // 查询K线数据
    BarListPtr barList = strategy->get_bar_list(code, period);
    if (!barList || barList->size() == 0) return false;
    
    // 填充数据
    int actual_count = min(count, (int)barList->size());
    for (int i = 0; i < actual_count; i++) {
        const WTSBarStruct& bar = barList->at(i);
        bars[i] = bar;
    }
    
    return true;
}
```

## 2.4 消息队列协作

### 2.4.1 事件推送机制

**WtMsgQue** - C++ 消息队列模块

**功能**：
- 基于 nanomsg 的消息队列实现
- 支持 PUB/SUB、REQ/REP 等模式
- 提供 C 接口供 Python 调用

**事件类型**：
- `TRD_TRADE`: 成交通知
- `TRD_ORDER`: 订单通知
- `TRD_NOTIFY`: 普通通知
- `LOG`: 日志通知

**C++端推送**：
```cpp
// 成交通知推送
void WtEngine::on_trade(const TradeInfo& trade) {
    WtMsgQue msgQue;
    msgQue.connect("tcp://127.0.0.1:3999");
    
    json msg;
    msg["strategy"] = trade.strategy_id;
    msg["code"] = trade.code;
    msg["qty"] = trade.qty;
    msg["price"] = trade.price;
    ...
    
    msgQue.publish("TRD_TRADE", msg.dump());
}
```

**Python端接收**：
```python
# WtMQWrapper.py
from wtpy import WtMsgQue, WtMQClient

class EventReceiver(WtMQClient):
    def __init__(self, url: str):
        mq = WtMsgQue()
        mq.add_mq_client(url, self)
        self.subscribe("TRD_TRADE")
        self.subscribe("TRD_ORDER")
    
    def on_mq_message(self, topic: str, message: str, dataLen: int):
        if topic == "TRD_TRADE":
            msg = json.loads(message)
            # 处理成交事件
            self.on_trade(msg)
```

## 2.5 扩展模块协作

### 2.5.1 扩展解析器

**C++端接口定义**：
```cpp
class IParserApi {
public:
    virtual bool init(WTSVariant* config) = 0;
    virtual void subscribe(const CodeSet& codes) = 0;
    virtual void registerSpi(IParserSpi* listener) = 0;
};
```

**Python端实现**：
```python
# ExtModuleDefs.py
class BaseExtParser:
    def on_tick(self, tick: dict):
        """Tick数据处理"""
        pass
    
    def on_bar(self, bar: dict):
        """K线数据处理"""
        pass

# 用户实现
class MyParser(BaseExtParser):
    def on_tick(self, tick: dict):
        # 处理Tick数据
        # 调用引擎接口推送数据
        self.engine.push_tick(tick)
```

**注册机制**：
```python
# WtDtWrapper.py
def create_ext_parser(self, parser_id: str) -> bool:
    """创建扩展解析器"""
    return self.api.create_ext_parser(parser_id.encode())

# WtDtEngine.py
def add_ext_parser(self, parser_id: str, parser: BaseExtParser):
    """添加扩展解析器"""
    # 注册到C++引擎
    self.__wrapper__.create_ext_parser(parser_id)
    
    # 保存Python对象引用
    self.__ext_parsers__[parser_id] = parser
```

### 2.5.2 扩展执行器

**工作机制类似**，通过 `BaseExtExecuter` 基类实现，注册到 C++ 引擎后，C++ 端会调用 Python 端的 `on_cmd()` 方法处理执行命令。

### 2.5.3 扩展数据加载器

**用于回测场景**，当策略需要加载外部数据时，C++ 引擎会调用 Python 端的 `load_bar()` 或 `load_tick()` 方法。

## 2.6 内存管理协作

### 2.6.1 引用计数机制

**C++端**：
- 使用智能指针管理对象生命周期
- 通过引用计数避免内存泄漏

**Python端**：
- Python 的垃圾回收机制自动管理
- 通过 `ctypes` 的 `byref()` 或指针传递避免数据拷贝


### 2.6.2 数据共享机制

**WtShareHelper** - 共享内存机制

**用途**：
- UFT 引擎使用共享内存实现进程间数据共享
- 降低进程间通信延迟

**C++端**：
```cpp
// 创建共享内存
WtShareHelper shareHelper;
shareHelper.create("MyData", sizeof(DataStruct));

// 写入数据
DataStruct* data = (DataStruct*)shareHelper.get();
data->price = 100.0;
```

**Python端**：
```python
# WtShareHelper.py
from wtpy.wrapper import WtDtHelper

helper = WtDtHelper()
helper.open_share("MyData")

# 读取数据
data = helper.read_share("MyData")
```

## 2.7 配置管理协作

### 2.7.1 配置文件加载

**Python端**：
```python
# WtEngine.py
def init(self, cfgfile: str = "config.yaml"):
    # 加载配置文件
    if cfgfile.endswith('.yaml'):
        with open(cfgfile, 'r', encoding='utf-8') as f:
            config = yaml.safe_load(f)
    else:
        with open(cfgfile, 'r', encoding='utf-8') as f:
            config = json.load(f)
    
    # 转换为JSON字符串传递给C++端
    config_str = json.dumps(config)
    self.__wrapper__.init_engine(config_str.encode(), False)
```

**C++端**：
```cpp
extern "C" bool init_engine(const char* cfg_json, bool isFile) {
    if (isFile) {
        // 从文件加载
        WTSVariant* config = WTSCfgLoader::load_from_file(cfg_json);
    } else {
        // 从JSON字符串加载
        WTSVariant* config = WTSCfgLoader::load_from_content(cfg_json);
    }
    
    // 初始化引擎
    engine->init(config);
    return true;
}
```

## 2.8 错误处理协作

### 2.8.1 异常传递机制

**C++端**：
- 使用返回码表示操作结果（true/false）
- 错误信息通过日志输出

**Python端**：
- 检查返回码，抛出异常
- 捕获异常并记录日志

**示例**：
```python
def enter_long(self, strategy_id: int, code: str, qty: float) -> bool:
    """开多仓接口"""
    ret = self.api.cta_enter_long(...)
    if not ret:
        # 记录错误日志
        self.log_error(f"Failed to enter long: {code}")
        return False
    return True
```

## 2.9 总结

WonderTrader 与 wtpy 的协作特点：

1. **清晰的接口边界**: C++ 核心通过 C 接口暴露功能，Python 层通过 ctypes 调用
2. **高效的数据传递**: 使用 NumPy 数组和 C 结构体直接映射，减少数据拷贝
3. **灵活的回调机制**: Python 策略通过回调函数与 C++ 引擎交互
4. **完善的扩展机制**: 支持扩展解析器、执行器、数据加载器等
5. **统一的消息队列**: 通过 WtMsgQue 实现事件推送和监控
6. **自动的内存管理**: C++ 使用智能指针，Python 使用垃圾回收

# 第三部分：基于 WonderTrader + wtpy 的完整开发流程

## 3.1 环境准备阶段

### 3.1.1 获取源码

**步骤1：克隆或下载源码**

```bash
# 克隆 WonderTrader C++ 核心
cd externalLib
git clone https://github.com/wondertrader/wondertrader.git

# 克隆 wtpy Python 框架
git clone https://github.com/wondertrader/wtpy.git
```

**目录结构**：
```
quant-learn/
├── externalLib/
│   ├── wondertrader/     # C++ 核心源码
│   │   ├── src/          # C++ 源代码
│   │   ├── docs/         # 文档
│   │   └── ...
│   └── wtpy/             # Python 框架
│       ├── wtpy/         # Python 源码
│       ├── demos/        # 示例代码
│       └── ...
```

### 3.1.2 编译 WonderTrader C++ 核心

**Windows 环境**：

1. **安装 Visual Studio**
   - 推荐 VS2017 或 VS2019
   - 安装 C++ 桌面开发工具集

2. **安装依赖库**
   - Boost 1.72+ (需要编译)
   - curl (需要编译)
   - 其他依赖见 `wondertrader/README.md`

3. **编译步骤**：
```bash
cd externalLib/wondertrader/src

# 方式1：使用 Visual Studio 打开解决方案
# - 打开 all.sln (全量编译)
# - 或打开对应的 .sln 文件（如 product.sln 用于实盘，backtest.sln 用于回测）

# 方式2：使用 CMake
mkdir build
cd build
cmake ..
cmake --build . --config Release
```

4. **确认编译输出**：
   - `WtPorter.dll` / `libWtPorter.so` (实盘引擎)
   - `WtBtPorter.dll` / `libWtBtPorter.so` (回测引擎)
   - `WtDtPorter.dll` / `libWtDtPorter.so` (数据引擎)
   - 其他动态库文件

**Linux 环境**：

1. **安装编译工具**：
```bash
sudo apt-get update
sudo apt-get install build-essential cmake
sudo apt-get install libboost-all-dev
sudo apt-get install libcurl4-openssl-dev
```

2. **编译步骤**：
```bash
cd externalLib/wondertrader/src
mkdir build && cd build
cmake ..
make -j4  # 使用4个线程并行编译
```

3. **复制动态库到 wtpy**：
```bash
# 运行脚本自动复制
cd externalLib/wondertrader
bash copy_bins_linux.sh
```

### 3.1.3 安装 Python 环境

**步骤1：创建虚拟环境**

```bash
# 使用 conda (推荐)
conda create -n wtpy python=3.8
conda activate wtpy

# 或使用 venv
python -m venv venv
source venv/bin/activate  # Linux/Mac
# 或
venv\Scripts\activate  # Windows
```

**步骤2：安装 wtpy**

**方式1：从 PyPI 安装**（推荐）
```bash
pip install wtpy --upgrade
```

**方式2：从本地源码安装**（开发模式）
```bash
cd externalLib/wtpy
pip install -e .
```

**方式3：直接使用源码**（无需安装）
```bash
# 设置 PYTHONPATH
export PYTHONPATH=$PYTHONPATH:/path/to/externalLib/wtpy
```

**步骤3：验证安装**

```python
import wtpy
print(wtpy.__version__)

# 测试导入核心模块
from wtpy import WtEngine, WtBtEngine, EngineType
print("wtpy 安装成功！")
```

### 3.1.4 配置基础数据文件

**准备基础配置文件**：

1. **复制示例配置**：
```bash
cd externalLib/wtpy/demos
cp -r common/ ../../your_project/
```

2. **配置品种信息** (`commodities.json`)：
```json
{
    "SHFE": {
        "rb": {
            "name": "螺纹钢",
            "exchg": "SHFE",
            "pricetick": 1.0,
            "volscale": 10
        }
    }
}
```

3. **配置合约信息** (`contracts.json`)：
```json
{
    "SHFE.rb.2305": {
        "code": "rb2305",
        "exchg": "SHFE",
        "product": "rb",
        "name": "螺纹钢2305"
    }
}
```

4. **配置交易时段** (`sessions.json`)：
```json
{
    "SHFE": {
        "name": "上期所",
        "sections": [
            {"begin": 900, "end": 1130},
            {"begin": 1330, "end": 1500},
            {"begin": 2100, "end": 2359}
        ]
    }
}
```

## 3.2 策略开发阶段

### 3.2.1 创建策略文件

**步骤1：创建策略目录结构**

```
your_project/
├── strategies/
│   └── MyStrategy.py      # 策略文件
├── configs/
│   ├── config.yaml        # 实盘配置
│   └── configbt.yaml      # 回测配置
├── common/                # 基础数据文件
│   ├── commodities.json
│   ├── contracts.json
│   └── sessions.json
└── run.py                 # 启动脚本
```

**步骤2：编写策略代码**

**示例：简单的双均线策略**

```python
# strategies/MyStrategy.py
from wtpy import BaseCtaStrategy, CtaContext

class MyStrategy(BaseCtaStrategy):
    """双均线策略"""
    
    def __init__(self, name: str):
        super().__init__(name)
        self.short_period = 20  # 短期均线周期
        self.long_period = 60   # 长期均线周期
    
    def on_init(self, context: CtaContext):
        """策略初始化"""
        # 订阅合约和K线
        context.stra_subscribe_ticks("SHFE.rb.2305")
        context.stra_reg_bar("SHFE.rb.2305", "m5", "MAINSECTION")
        
        # 输出日志
        context.stra_log_text("策略初始化完成")
    
    def on_calculate(self, context: CtaContext):
        """策略计算（K线闭合时触发）"""
        code = "SHFE.rb.2305"
        
        # 获取K线数据
        bars = context.stra_get_bars(code, "m5", self.long_period)
        if bars.size() < self.long_period:
            return
        
        # 计算均线
        close_prices = bars.close
        short_ma = close_prices[-self.short_period:].mean()
        long_ma = close_prices[-self.long_period:].mean()
        
        # 获取当前持仓
        pos = context.stra_get_position(code)
        
        # 交易逻辑
        if short_ma > long_ma and pos == 0:
            # 金叉且无持仓，开多
            context.stra_enter_long(code, 1, "金叉开多")
            context.stra_log_text(f"金叉开多，短期均线={short_ma:.2f}, 长期均线={long_ma:.2f}")
        
        elif short_ma < long_ma and pos > 0:
            # 死叉且持有多仓，平多
            context.stra_exit_long(code, 1, "死叉平多")
            context.stra_log_text(f"死叉平多，短期均线={short_ma:.2f}, 长期均线={long_ma:.2f}")
```

### 3.2.2 编写回测脚本

**回测启动脚本** (`runBT.py`)：

```python
# runBT.py
from wtpy import WtBtEngine, EngineType
from strategies.MyStrategy import MyStrategy

if __name__ == '__main__':
    # 创建回测引擎
    engine = WtBtEngine(EngineType.ET_CTA)
    
    # 初始化引擎
    engine.init("configs/configbt.yaml")
    
    # 添加策略
    strategy = MyStrategy("MyStrategy")
    engine.add_strategy(strategy)
    
    # 配置回测参数
    engine.config_backtest(
        start_date=20220101,   # 开始日期
        end_date=20221231,     # 结束日期
        initial_capital=1000000,  # 初始资金
        slippage=2            # 滑点（跳）
    )
    
    # 运行回测
    engine.run_backtest()
    
    # 输出回测结果
    print("回测完成！")
```

**回测配置文件** (`configs/configbt.yaml`)：

```yaml
basefiles:
  commodity: "common/commodities.json"
  contract: "common/contracts.json"
  holiday: "common/holidays.json"
  session: "common/sessions.json"

data:
  loader: "csv"  # 或 "bin"
  path: "storage/history"  # 历史数据路径

# 策略配置
strategies:
  - name: MyStrategy
    active: true
    params:
      short_period: 20
      long_period: 60
```

### 3.2.3 准备历史数据

**方式1：使用 datahelper 下载数据**

```python
# download_data.py
from wtpy.apps.datahelper import DHRqData

# 创建数据助手
helper = DHRqData()

# 下载K线数据
helper.dumpBars(
    codes=["SHFE.rb.2305"],
    start_date="20220101",
    end_date="20221231",
    period="m5",
    output="storage/history"
)
```

**方式2：手动准备CSV格式数据**

CSV格式要求：
```csv
date,time,open,high,low,close,volume,turnover,open_interest
20220104,900,4500,4520,4490,4510,1000,4510000,10000
20220104,905,4510,4525,4505,4520,1200,5424000,12000
...
```

文件命名：`SHFE.rb.2305.m5.csv`

## 3.3 回测验证阶段

### 3.3.1 运行回测

```bash
python runBT.py
```

### 3.3.2 分析回测结果

**使用 WtBtAnalyst 分析**：

```python
# analyze_backtest.py
from wtpy.apps import WtBtAnalyst

# 创建分析器
analyst = WtBtAnalyst()

# 分析回测结果
analyst.run(
    folder="outputs_bt/MyStrategy",  # 回测输出目录
    period="m5",
    init_capital=1000000,
    rf_rate=0.03  # 无风险利率
)

# 生成Excel报告
analyst.gen_report("backtest_report.xlsx")
```

**回测指标说明**：

- **收益指标**：
  - 总收益率
  - 年化收益率
  - 累计净值

- **风险指标**：
  - 最大回撤
  - 波动率
  - 下行波动率

- **风险调整收益**：
  - Sharpe比率
  - Sortino比率
  - Calmar比率

- **交易统计**：
  - 胜率
  - 盈亏比
  - 平均持仓时间

### 3.3.3 参数优化

**使用 WtCtaOptimizer 进行参数优化**：

```python
# optimize_params.py
from wtpy.apps import WtCtaOptimizer
from strategies.MyStrategy import MyStrategy

# 创建优化器
optimizer = WtCtaOptimizer()

# 添加策略
optimizer.add_strategy(
    "MyStrategy",
    MyStrategy,
    {
        "short_period": [10, 15, 20, 25, 30],
        "long_period": [40, 50, 60, 70, 80]
    }
)

# 配置优化参数
optimizer.set_gencfg(
    start_date=20220101,
    end_date=20221231,
    initial_capital=1000000
)

# 运行优化
optimizer.optimize(
    output="optimize_results.csv",
    workers=4  # 并行进程数
)
```

**优化结果分析**：

优化完成后，查看 `optimize_results.csv`，选择最优参数组合：
- 综合考虑：收益率、最大回撤、Sharpe比率
- 避免过拟合：使用样本外数据验证

## 3.4 实盘部署阶段

### 3.4.1 编写实盘配置

**实盘配置文件** (`configs/config.yaml`)：

```yaml
basefiles:
  commodity: "common/commodities.json"
  contract: "common/contracts.json"
  holiday: "common/holidays.json"
  session: "common/sessions.json"

# 行情解析器配置
parsers:
  - name: parser1
    module: ParserCTP
    front: "tcp://180.168.146.187:10131"
    broker: "9999"
    user: "your_username"
    pass: "your_password"

# 交易通道配置
traders:
  - name: trader1
    module: TraderCTP
    front: "tcp://180.168.146.187:10130"
    broker: "9999"
    user: "your_username"
    pass: "your_password"
    appid: "simnow_client_test"
    authcode: "0000000000000000"

# 执行器配置
executers:
  - name: executer1
    module: WtLocalExecuter
    session: "MAINSECTION"
    active: true

# 策略配置
strategies:
  - name: MyStrategy
    active: true
    params:
      short_period: 20
      long_period: 60
```

### 3.4.2 编写实盘启动脚本

**实盘启动脚本** (`run.py`)：

```python
# run.py
from wtpy import WtEngine, EngineType
from strategies.MyStrategy import MyStrategy

if __name__ == '__main__':
    # 创建实盘引擎
    engine = WtEngine(EngineType.ET_CTA)
    
    # 初始化引擎
    engine.init("configs/config.yaml")
    
    # 添加策略
    strategy = MyStrategy("MyStrategy")
    engine.add_cta_strategy(strategy, 
        contracts=["SHFE.rb.2305"],
        session="MAINSECTION"
    )
    
    # 提交配置
    engine.commit_config()
    
    # 运行引擎
    engine.run()
    
    # 等待用户中断
    input("按回车键停止...")
    engine.stop()
```

### 3.4.3 启动监控服务

**监控服务启动脚本** (`run_monitor.py`)：

```python
# run_monitor.py
from wtpy.monitor import WtMonSvr

if __name__ == '__main__':
    # 创建监控服务
    monitor = WtMonSvr("monitor.yaml")
    
    # 启动服务
    monitor.run()
```

**监控服务配置** (`monitor.yaml`)：

```yaml
host: "0.0.0.0"
port: 8080
grpcfg: "groups"  # 组合盘配置目录

# 消息队列配置
mqurl: "tcp://127.0.0.1:3999"
```

**访问监控界面**：

浏览器打开：`http://localhost:8080`

功能：
- 查看策略运行状态
- 查看资金、持仓、订单
- 手动启动/停止策略
- 查看实时日志

### 3.4.4 模拟交易测试

**使用 TraderMocker 进行模拟交易**：

```yaml
# config.yaml 中配置模拟交易
traders:
  - name: trader1
    module: TraderMocker
    active: true
```

模拟交易特点：
- 使用真实行情数据
- 本地撮合，不连接交易所
- 可以测试策略逻辑
- 不会产生真实交易

## 3.5 运维管理阶段

### 3.5.1 使用 WatchDog 进程守护

**WatchDog 配置** (`monitor.yaml`)：

```yaml
watchdog:
  enabled: true
  check_interval: 60  # 检查间隔（秒）
  auto_restart: true   # 自动重启
  
  tasks:
    - name: "MyStrategy"
      cmd: "python run.py"
      working_dir: "/path/to/your_project"
      schedule:
        - day: "monday"
          time: "09:00"
        - day: "tuesday"
          time: "09:00"
        # ...
```

### 3.5.2 日志管理

**日志配置** (`logcfg.yaml`)：

```yaml
writers:
  - name: console
    type: console
    level: info
    
  - name: file
    type: file
    path: logs/wtpy.log
    level: debug
    daily: true
    reserve: 7  # 保留7天日志
```

**日志级别**：
- `debug`: 调试信息
- `info`: 一般信息
- `warning`: 警告信息
- `error`: 错误信息

### 3.5.3 性能监控

**监控指标**：

1. **策略性能**：
   - 资金曲线
   - 持仓变化
   - 交易统计

2. **系统性能**：
   - CPU使用率
   - 内存使用率
   - 网络延迟

3. **数据质量**：
   - 行情接收延迟
   - 数据完整性
   - 异常数据统计

### 3.5.4 风险控制

**风控配置** (`filters.yaml`)：

```yaml
fund:
  max_drawdown: 0.20  # 最大回撤20%
  max_loss: 50000     # 最大亏损

position:
  max_single: 10      # 单个合约最大持仓
  max_total: 50       # 总持仓限制

order:
  max_cancel_count: 100  # 最大撤单次数
  max_order_count: 1000   # 最大下单次数
```

## 3.6 持续改进阶段

### 3.6.1 定期回测验证

**定期回测流程**：

1. **每周回测**：
   - 使用最新数据
   - 验证策略表现
   - 检查参数是否过拟合

2. **每月深度分析**：
   - 多周期回测
   - 不同市场环境测试
   - 参数敏感性分析

3. **季度策略评估**：
   - 策略有效性评估
   - 市场适应性分析
   - 是否需要调整或下线

### 3.6.2 策略迭代

**迭代流程**：

1. **发现问题**：
   - 回测表现不佳
   - 实盘表现与回测差异大
   - 市场环境变化

2. **改进策略**：
   - 优化交易逻辑
   - 调整参数
   - 增加风控规则

3. **验证改进**：
   - 回测验证
   - 模拟交易测试
   - 小资金实盘测试

4. **正式部署**：
   - 全资金实盘
   - 持续监控

### 3.6.3 版本管理

**版本控制建议**：

1. **代码版本**：
   - 使用 Git 管理代码
   - 为每个版本打标签
   - 记录变更日志

2. **配置版本**：
   - 配置文件版本化
   - 记录配置变更原因
   - 保留历史配置

3. **数据版本**：
   - 历史数据版本管理
   - 回测结果归档
   - 性能报告保存

## 3.7 常见问题排查

### 3.7.1 动态库加载失败

**问题**：`OSError: [WinError 126] 找不到指定的模块`

**解决方案**：
1. 检查动态库路径是否正确
2. 检查依赖库是否缺失（如 VC++ 运行库）
3. 使用 `dependency walker` 检查依赖

### 3.7.2 策略回调不触发

**问题**：策略的 `on_calculate()` 不触发

**解决方案**：
1. 检查是否订阅了K线
2. 检查K线周期是否正确
3. 检查历史数据是否完整
4. 检查日志输出

### 3.7.3 数据查询返回空

**问题**：`stra_get_bars()` 返回空数据

**解决方案**：
1. 检查合约代码格式是否正确
2. 检查K线周期是否匹配
3. 检查历史数据文件是否存在
4. 检查数据格式是否正确

### 3.7.4 交易接口连接失败

**问题**：无法连接到交易接口

**解决方案**：
1. 检查网络连接
2. 检查交易接口地址和端口
3. 检查账号密码是否正确
4. 检查防火墙设置

## 3.8 最佳实践总结

### 3.8.1 策略开发建议

1. **代码规范**：
   - 使用类型提示
   - 添加注释说明
   - 遵循PEP 8规范

2. **错误处理**：
   - 添加异常捕获
   - 记录错误日志
   - 优雅降级处理

3. **性能优化**：
   - 避免频繁数据查询
   - 使用数据缓存
   - 减少不必要的计算

### 3.8.2 回测建议

1. **数据质量**：
   - 使用高质量历史数据
   - 检查数据完整性
   - 处理数据异常

2. **回测设置**：
   - 考虑滑点和手续费
   - 使用真实交易时段
   - 避免未来函数

3. **结果分析**：
   - 多角度分析结果
   - 关注最大回撤
   - 避免过拟合

### 3.8.3 实盘部署建议

1. **风险控制**：
   - 设置合理止损
   - 控制单笔交易风险
   - 限制总持仓规模

2. **监控告警**：
   - 设置资金回撤告警
   - 监控异常交易
   - 记录关键事件

3. **应急预案**：
   - 准备手动停止流程
   - 准备数据备份方案
   - 准备故障恢复方案

---

## 完整开发流程图

```
┌─────────────────┐
│  环境准备阶段    │
│ 1. 获取源码     │
│ 2. 编译C++核心  │
│ 3. 安装Python   │
│ 4. 配置基础数据 │
└────────┬────────┘
         ↓
┌─────────────────┐
│  策略开发阶段    │
│ 1. 编写策略代码 │
│ 2. 准备历史数据 │
│ 3. 编写回测脚本 │
└────────┬────────┘
         ↓
┌─────────────────┐
│  回测验证阶段    │
│ 1. 运行回测     │
│ 2. 分析结果     │
│ 3. 参数优化     │
└────────┬────────┘
         ↓
┌─────────────────┐
│  实盘部署阶段    │
│ 1. 编写实盘配置 │
│ 2. 启动交易引擎 │
│ 3. 启动监控服务 │
└────────┬────────┘
         ↓
┌─────────────────┐
│  运维管理阶段    │
│ 1. 进程守护     │
│ 2. 日志管理     │
│ 3. 性能监控     │
└────────┬────────┘
         ↓
┌─────────────────┐
│  持续改进阶段    │
│ 1. 定期回测     │
│ 2. 策略迭代     │
│ 3. 版本管理     │
└─────────────────┘
```

---

## 总结

本文档详细阐述了：

1. **wtpy 完整框架体系**：从架构设计到核心模块，从数据流转到性能特性
2. **WonderTrader 与 wtpy 协作机制**：从接口对接到数据流转，从扩展机制到内存管理
3. **完整开发流程**：从环境准备到策略开发，从回测验证到实盘部署，从运维管理到持续改进

希望本文档能够帮助您全面理解和使用 WonderTrader + wtpy 量化交易框架，顺利完成策略开发和实盘部署。